In [1]:
from datasets import load_from_disk

wiki_datasets = load_from_disk("/home/ubuntu/workspace/data/wiki_preprocessed_droped")
train_dataset = load_from_disk("/home/ubuntu/workspace/data/new_train_doc")
wiki_datasets.load_elasticsearch_index("text", host="localhost", port="9200", es_index_name="wikipedia_contexts")

In [2]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
        num_rows: 3351
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
        num_rows: 841
    })
})

In [26]:
import pandas as pd

data = train_dataset['train'].to_pandas()

In [37]:
answers = pd.read_csv('d.csv')
answers

,id,result1,result2,result3,result4,result5,total
0,mrc-0-003264,True,NaN,True,True,True,4.0
1,mrc-0-005256,True,NaN,True,True,True,4.0
2,mrc-0-003360,False,NaN,False,True,False,1.0
3,mrc-1-001432,True,NaN,False,False,False,1.0
4,mrc-0-003506,True,NaN,True,True,True,4.0
...,...,...,...,...,...,...,...
3346,mrc-0-000199,NaN,True,True,True,True,4.0
3347,mrc-0-001646,NaN,True,True,False,True,3.0
3348,mrc-0-000073,NaN,True,True,True,True,4.0
3349,mrc-0-003890,NaN,True,True,True,True,4.0


In [27]:
data

,__index_level_0__,answers,context,document_id,id,question,title,chunks
0,572,"{'answer_start': [1021], 'text': ['1890년 5월 18...","중서서원에서 4년간 공부하며 윤치호는 개신교 선교사들의 영향을 받게 되었고, 중서서...",725,mrc-0-000857,윤치호가 조선이 문명국의 식민지가 되는 것이 낫다고 한 날은?,윤치호,"[중서서원에서 4년간 공부하며 윤치호는 개신교 선교사들의 영향을 받게 되었고, 중서..."
1,3210,"{'answer_start': [9], 'text': ['무역']}",란돌포는 부자로 무역을 하기 위해 큰 배에 전재산을 털어 상품을 사서 키프로스 섬으...,25374,mrc-0-004943,란돌포가 전 재산을 투자한 상품으로 하고자 한 일은?,데카메론의 줄거리,[란돌포는 부자로 무역을 하기 위해 큰 배에 전재산을 털어 상품을 사서 키프로스 섬...
2,3257,"{'answer_start': [761], 'text': ['1934년']}",1927년 고등학교 학생의 신분으로 실업 팀에 선발되어 친선 경기를 위해 일본을 방...,36586,mrc-0-005013,와카바야시의 추계 리그 3번째 우승은 몇 년인가?,와카바야시 다다시,[1927년 고등학교 학생의 신분으로 실업 팀에 선발되어 친선 경기를 위해 일본을 ...
3,754,"{'answer_start': [174], 'text': ['임금']}","보상과 관련하여, 우선 '보상공정성'이 직무만족도에 영향을 주는 중요한 요인이다. ...",46589,mrc-1-001144,보상제도에 있어 가장 기본이 되는 것은?,직무 만족,"[보상과 관련하여, 우선 '보상공정성'이 직무만족도에 영향을 주는 중요한 요인이다...."
4,805,"{'answer_start': [398], 'text': ['조지 호수']}","1757년 로던 경이 세운 작전은 1756년 9월에 영국 본국 정부에 제출되었고, ...",34592,mrc-1-001234,이안 스틸이 군에 최적화된 수로라고 말한 호수는?,윌리엄 헨리 요새 전투,"[1757년 로던 경이 세운 작전은 1756년 9월에 영국 본국 정부에 제출되었고,..."
...,...,...,...,...,...,...,...,...
3346,2034,"{'answer_start': [202], 'text': ['1948년']}",1945년 8·15 광복 직후 한국으로 귀국하였고 이화여자중학교에 피겨 특기생으로 ...,24094,mrc-0-003087,한국에서 처음으로 여자 피겨 선수권 대회 1등을 배출한 해는?,홍용명,[1945년 8·15 광복 직후 한국으로 귀국하였고 이화여자중학교에 피겨 특기생으로...
3347,2226,"{'answer_start': [875], 'text': ['윤덕원']}",어린 시절 음반 콜렉터였던 삼촌을 통해 음악을 많이 듣게 되었다. 고등학교 때 취미...,44767,mrc-0-003393,술탄이라는 이름의 아이디어를 제공한 사람은?,나잠 수,[어린 시절 음반 콜렉터였던 삼촌을 통해 음악을 많이 듣게 되었다. 고등학교 때 취...
3348,254,"{'answer_start': [1531], 'text': ['미국 대통령 버락 오...",빌리 진은 1965년 9월 17일 캘리포니아 주 롱비치에서 래리 킹과 결혼하였다. ...,20430,mrc-1-000393,빌리 진 킹은 누구 덕분에 미국 대표단의 대표로 올림픽에 참가할 수 있었나?,빌리 진 킹,[빌리 진은 1965년 9월 17일 캘리포니아 주 롱비치에서 래리 킹과 결혼하였다....
3349,721,"{'answer_start': [170], 'text': ['스타다 코난족']}",전쟁의 기원에 대한 기록은 1540년대에 자크 카르티에가 세인트 로렌스 골짜기를 항...,15206,mrc-0-001089,토우다만 족으로 알려진 종족과 싸운 것은 누구인가?,비버 전쟁,[전쟁의 기원에 대한 기록은 1540년대에 자크 카르티에가 세인트 로렌스 골짜기를 ...


In [40]:

from sklearn.model_selection import StratifiedKFold

def get_stratified_K_fold(dataset: pd.DataFrame, num):
    """
    stratified_K_fold를 구현하였습니다.
    n_splits은 몇개로 나눌것인지를 의미합니다.
    num은 n_splits으로 나눴을때 몇번째 데이터를 가져올지를 의미합니다.
    shuffle은 랜덤하게 나눕니다. 이때는 num을 사용하지않고 무조건 1번째것을 가져옵니다.
    """
    skf = StratifiedKFold(4)
    k_fold_data = list(skf.split(dataset, dataset["total"]))
    train_index, valid_index = (
        k_fold_data[num][0],
        k_fold_data[num][1],
    )
    return dataset.iloc[train_index], dataset.iloc[valid_index]


In [49]:
_, result1 = get_stratified_K_fold(answers, 0)
_, result2 = get_stratified_K_fold(answers, 1)
_, result3 = get_stratified_K_fold(answers, 2)
_, result4 = get_stratified_K_fold(answers, 3)

In [52]:
valid1_id = list(result1['id'])
valid2_id = list(result2['id'])
valid3_id = list(result3['id'])
valid4_id = list(result4['id'])

In [75]:
def split_train_valid(datasets, valid, valid_id):
  valid_dataset = datasets.filter(lambda x: x['id'] in valid_id)
  train_dataset = datasets.filter(lambda x: x['id'] not in valid_id)
  columns = ['context', 'question', 'id', 'title', 'document_id', 'answers']
  train_datadict = {}
  for col in columns:
    train_datadict[col] = []
  for data in train_dataset:
    for column in columns:
      train_datadict[column].append(data[column])
  for data in valid:
    for column in columns:
      train_datadict[column].append(data[column])
  return train_dataset, valid_dataset

In [84]:
result = split_train_valid(train_dataset['train'], train_dataset['validation'], valid4_id)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [85]:
result

(Dataset({
     features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
     num_rows: 2514
 }),
 Dataset({
     features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
     num_rows: 837
 }))

In [86]:
import random

hard_sample = []

def ds_data_function(data):
  query = data['question']
  negative_contexts = []
  _, retrieved_examples = wiki_datasets.get_nearest_examples("text", query, k=100)
  if data['document_id'] not in retrieved_examples['document_id']:
    hard_sample.append([data, retrieved_examples])
  for index in range(50):
    if retrieved_examples['document_id'][index] == data['document_id']:
      continue
    negative_contexts.append(retrieved_examples['text'][index])
    if len(negative_contexts) == 9:
      break
  random.shuffle(negative_contexts)
  index = random.randint(0, 9)
  negative_contexts.insert(index, data['context'])
  answer_index = data['answers']['answer_start'][0]
  for negative_index in range(index):
    answer_index += len(negative_contexts[negative_index])
  data['answers']['answer_start'][0] = answer_index + index
  data['context'] = " ".join(negative_contexts)
  return data

In [87]:
new_train_dataset = result[0].map(ds_data_function, num_proc=4)
new_valid_dataset = result[1].map(ds_data_function, num_proc=4)

/home/ubuntu/workspace/mrc_venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/home/ubuntu/workspace/mrc_venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/home/ubuntu/workspace/mrc_venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not e

In [88]:
len(hard_sample)

0

In [89]:
search_error = new_train_dataset.filter(lambda example: example['context'][example['answers']['answer_start'][0]:example['answers']['answer_start'][0]+len(example['answers']['text'][0])] != example['answers']['text'][0])

  0%|          | 0/3 [00:00<?, ?ba/s]

In [90]:
search_error

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
    num_rows: 0
})

In [91]:
from datasets import DatasetDict

data = {
  'train': new_train_dataset,
  'validation': train_dataset['validation']
}
train = DatasetDict(data)
train.save_to_disk('new_ds_train_datasets5') # 저장위치